In [2]:
# import libraries
import sqlite3
import pandas as pd
import os
import ast
from datetime import datetime

In [3]:
# setup sqlite connection
con = sqlite3.connect('../airflow/airflow.db')
cur = con.cursor()

In [6]:
import_list = pd.read_sql_query("Select distinct [Visitor/Neutral] from games2;", con)
match_list = pd.read_sql_query('Select * from teams;', con)

In [7]:
import_list

,Visitor/Neutral
0,Golden State Warriors
1,Los Angeles Clippers
2,Charlotte Hornets
3,New York Knicks
4,Miami Heat
5,Washington Wizards
6,New Orleans Pelicans
7,Milwaukee Bucks
8,Atlanta Hawks
9,San Antonio Spurs


In [44]:
import pandas as pd
import requests

In [81]:
date_df = pd.read_sql_query("SELECT a.date, a.arena, b.city, b.state FROM games a join arena b on a.arena = b.arena where a.date <= '2022-03-07';", con)


In [82]:
date_df.describe()

,date,Arena,City,State
count,2144,2144,2144,2144
unique,323,29,29,23
top,2021-05-16,Crypto.com Arena,Los Angeles,California
freq,15,154,154,294


In [83]:
check_list = list(date_df.to_dict('index').values())

In [39]:
check_list

[{'date': '2020-12-22',
  'Arena': 'Barclays Center',
  'City': 'Brooklyn',
  'State': 'New York'},
 {'date': '2020-12-22',
  'Arena': 'Crypto.com Arena',
  'City': 'Los Angeles',
  'State': 'California'},
 {'date': '2020-12-23',
  'Arena': 'Rocket Mortgage FieldHouse',
  'City': 'Cleveland',
  'State': 'Ohio'},
 {'date': '2020-12-23',
  'Arena': 'Gainbridge Fieldhouse',
  'City': 'Indianapolis',
  'State': 'Indiana'},
 {'date': '2020-12-23',
  'Arena': 'Amway Center',
  'City': 'Orlando',
  'State': 'Florida'},
 {'date': '2020-12-23',
  'Arena': 'Wells Fargo Center',
  'City': 'Philadelphia',
  'State': 'Pennsylvania'},
 {'date': '2020-12-23',
  'Arena': 'Scotiabank Arena',
  'City': 'Toronto',
  'State': 'Ontario'},
 {'date': '2020-12-23',
  'Arena': 'TD Garden',
  'City': 'Boston',
  'State': 'Massachusetts'},
 {'date': '2020-12-23',
  'Arena': 'United Center',
  'City': 'Chicago',
  'State': 'Illinois'},
 {'date': '2020-12-23',
  'Arena': 'FedExForum',
  'City': 'Memphis',
  'State

In [31]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [85]:
# covid data
counter= 0
for game in check_list:

    try:
        if game['State'] == 'Ontario':
            url = f'https://api.opencovid.ca/timeseries?stat=cases&loc=prov&date={game["date"]}'
            r=requests.get(url)
            j=r.json()['cases']
            clean = list(filter(lambda data: data['province'] == 'Ontario', j))[0]
            game['cases'] = clean['cases']
            game['total'] = clean['cumulative_cases']
        else:
            if game['State'] == 'D.C.':
                url = f"https://api.covidtracking.com/v2/states/dc/{game['date']}.json"
                r = requests.get(url)
                j = r.json()['data']['cases']['total']
                game['total'] = j['value']
                game['cases'] = j['calculated']['change_from_prior_day']
                game['seven_day_change_percent'] = j['calculated']['seven_day_change_percent']
            else:
                url = f"https://api.covidtracking.com/v2/states/{(us_state_to_abbrev[game['State']]).lower()}/{game['date']}.json"
                r = requests.get(url)
                j = r.json()['data']['cases']['total']
                game['total'] = j['value']
                game['cases'] = j['calculated']['change_from_prior_day']
                game['seven_day_change_percent'] = j['calculated']['seven_day_change_percent']
    except KeyError:
        counter +=1
    
print(counter)

1559


In [86]:
check_list

[{'date': '2020-12-22',
  'Arena': 'Barclays Center',
  'City': 'Brooklyn',
  'State': 'New York',
  'total': 866765,
  'cases': 9716,
  'seven_day_change_percent': 9.1},
 {'date': '2020-12-22',
  'Arena': 'Crypto.com Arena',
  'City': 'Los Angeles',
  'State': 'California',
  'total': 1925007,
  'cases': 32659,
  'seven_day_change_percent': 19},
 {'date': '2020-12-23',
  'Arena': 'Rocket Mortgage FieldHouse',
  'City': 'Cleveland',
  'State': 'Ohio',
  'total': 644822,
  'cases': 7790,
  'seven_day_change_percent': 10.3},
 {'date': '2020-12-23',
  'Arena': 'Gainbridge Fieldhouse',
  'City': 'Indianapolis',
  'State': 'Indiana',
  'total': 476538,
  'cases': 4662,
  'seven_day_change_percent': 8.1},
 {'date': '2020-12-23',
  'Arena': 'Amway Center',
  'City': 'Orlando',
  'State': 'Florida',
  'total': 1213760,
  'cases': 11100,
  'seven_day_change_percent': 6.8},
 {'date': '2020-12-23',
  'Arena': 'Wells Fargo Center',
  'City': 'Philadelphia',
  'State': 'Pennsylvania',
  'total': 58

In [87]:
upload_draft = pd.DataFrame(check_list)

In [88]:
upload_draft

,date,Arena,City,State,total,cases,seven_day_change_percent
0,2020-12-22,Barclays Center,Brooklyn,New York,866765.0,9716.0,9.1
1,2020-12-22,Crypto.com Arena,Los Angeles,California,1925007.0,32659.0,19.0
2,2020-12-23,Rocket Mortgage FieldHouse,Cleveland,Ohio,644822.0,7790.0,10.3
3,2020-12-23,Gainbridge Fieldhouse,Indianapolis,Indiana,476538.0,4662.0,8.1
4,2020-12-23,Amway Center,Orlando,Florida,1213760.0,11100.0,6.8
...,...,...,...,...,...,...,...
2139,2021-01-31,Ball Arena,Denver,Colorado,396179.0,1163.0,2.6
2140,2021-01-31,Gainbridge Fieldhouse,Indianapolis,Indiana,626682.0,1723.0,2.6
2141,2021-01-31,Scotiabank Arena,Toronto,Ontario,272331.0,1831.0,NaN
2142,2021-01-31,Capital One Arena,Washington,D.C.,36872.0,210.0,4.5


In [89]:
upload_clean = upload_draft[['date', 'City', 'State', 'total', 'cases', 'seven_day_change_percent']]

In [90]:
upload_clean.to_sql(name = 'cases', con=con, index=False, if_exists='append')

In [91]:
# finalize and close connections
con.commit()
cur.close()
con.close()